In [56]:
from sympy import *
init_printing(use_unicode=True)

In [237]:
import gc
gc.collect()

In [58]:
x = Symbol('x', commutative=False)
y = Symbol('y', commutative=False)
X = Symbol('X', commutative=False)
Y = Symbol('Y', commutative=False)
eps = Symbol('\epsilon')

In [60]:
def comm(a,b):
    return a*b*a**-1*b**-1

In [88]:
_series_dict = {}
_magnus_dict = {}
def magnus(w):
    if w in _magnus_dict:
        return _magnus_dict[w]
    
    expr = (w.subs(x,1+eps*X).subs(y,1+eps*Y) - 1).series(eps, n=None)
    return expr

In [168]:
w = comm(x,y)

In [165]:
((1 + eps*X)**-1).series?

In [ ]:
((1 + eps*X)**-1).series

In [242]:
def magnus(w, n=None):
    srs = (w.subs({x:1+eps*X, y:1+eps*Y}) - 1).series(eps,n = None if n is None else n+1)
    if n is None:
        return next(srs).subs(eps,1)
    return srs.removeO() / eps**n

In [245]:
w = comm(w,y)

In [247]:
%timeit magnus(w,3)
%timeit magnus(w)

689 ms ± 7.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
849 ms ± 6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [178]:
Expr.subs?

In [180]:
theta = lambda expr : expr.subs({x:x**-1, y:y**-1})

In [186]:
w = comm(w,theta(w))
w

In [188]:
a = [x]
b = [y]
for n in range(20):
    a_ = a[-1]
    b_ = b[-1]
    a.append(a_ * b_)
    b.append(a_**-1 * b_**-1)

In [223]:
n=3
a[n],b[n]

Claim:
a[n] = x..x if n == 0 (3)
a[n] = x..y if n == 1 (3)
a[n] = x..y^-1 if n == 2 (3)
and
b[n] = y..y if n == 0 (3)
b[n] = x^-1..y^-1 if n == 1 (3)
b[n] = y^-1..x if n == 2 (3)
and there is never any cancellation

In [232]:
for n in range(10):
    %timeit -n1 -r1 print(magnus(a[n]))

X
18.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
X + Y
84 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
X*Y - Y*X
191 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
-2*X*Y*X - X*Y**2 + X**2*Y + 2*Y*X*Y + Y*X**2 - Y**2*X
6.16 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
-4*X*Y*X*Y*X - X*Y*X*Y**2 + 3*X*Y*X**2*Y + 3*X*Y**2*X*Y + X*Y**2*X**2 - 2*X*Y**3*X - X**2*Y*X*Y + X**2*Y**2*X - 4*Y*X*Y*X*Y - Y*X*Y*X**2 + 3*Y*X*Y**2*X + 3*Y*X**2*Y*X + Y*X**2*Y**2 - 2*Y*X**3*Y - Y**2*X*Y*X + Y**2*X**2*Y
28min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


KeyboardInterrupt: 

In [236]:
import numpy as np
np.log((84/18.1, 191/84, 6.16*1000 / 191, 28*60/6.16)) / np.log(2)

array([ 2.21439963,  1.18511141,  5.01128581,  8.09131517])

In [38]:
def bch(w, n):
    expr = w.subs(x,exp(eps*X)).subs(y,exp(eps*Y)).series(eps, n=n+1) - 1
    return expr.removeO() / eps**n

In [65]:
%timeit magnus(w,2)
%timeit bch(w,2)

167 ms ± 64.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
546 ms ± 63.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
def degree(w):
    if w == 1:
        return 0 # should be inf actually
    d = 1
    while magnus(w,d) == 0:
        d += 1
    return d

In [14]:
def degree_var(w):
    if w == 1:
        return 0
    d = 1
    while magnus_var(w,d) == 0:
        d += 1
    return d

In [15]:
def decode_word(n):
    letters = [x, y, x**-1, y**-1]
    w = x
    i = 0 # index of letter
    while n > 0:
        i = (i + n % 3 - 1) % 4 # change by 1, 0, or -1
        w *= letters[i]
        n //= 3 
    return w

In [16]:
def length_word(n):
    count = 0
    while n > 0:
        n //= 3
        count += 1
    return count

In [19]:
w = comm(comm(x,y),y)
magnus_var(w,3)

In [20]:
%timeit magnus(w,3)
%timeit magnus_var(w,3)

929 ms ± 45.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
525 µs ± 5.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
%timeit degree_var(decode_word(10))

302 µs ± 47.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit degree(decode_word(10))

108 ms ± 2.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [156]:
for n in log_progress(range(1000), every=1):
    magnus(decode_word(n))

A Jupyter Widget

KeyboardInterrupt: 

In [28]:
for n in log_progress(range(1000), every=10):
    degree(decode_word(n))

A Jupyter Widget

KeyboardInterrupt: 

In [319]:
from log_progress import log_progress

alpha = {}
for n in log_progress(range(1000000), every=1000):
    if n not in deg_dict:
        deg_dict[n] = degree_var(decode_word(n))
    l = length_word(n)
    d = deg_dict[n]
    if d not in alpha:
        alpha[d] = l
        print(d, l, decode_word(n))

A Jupyter Widget

1 0 x
2 3 x*y*x**(-1)*y**(-1)
3 7 x*y*x**(-2)*y*x*y**(-2)


KeyboardInterrupt: 